# Producer

In [37]:
from kafka import KafkaProducer
import json
import time

# Emission data
emission_data = [{"2000": 7.3015565276, "2001": 7.3445498509, "2002": 7.5910820191, "2003": 7.6661065819,
                  "2004": 7.9264551349, "2005": 8.022971702, "2006": 8.247301388, "2007": 8.3558799857,
                  "2008": 8.4741724795, "2009": 8.4774061324, "2010": 8.3759737748, "2011": 8.4235682823,
                  "2012": 8.7662938718, "2013": 9.291035915, "2014": 9.8128903391, "2015": 10.1173860799,
                  "2016": 10.4965465177, "2017": 10.8414736715, "2018": 10.6853845334, "2019": 10.6693128165,
                  "2020": 10.9580914025, "2021": 11.1264878349, "2022": 11.0597619728}]

# Kafka topic name
topic_name = "emissions"

# Kafka producer configuration
producer = KafkaProducer(
    bootstrap_servers=['172.29.16.101:9092'],
    value_serializer=lambda x: json.dumps(x).encode('utf-8')
)

def send_to_kafka(producer, topic, data):
    producer.send(topic, value=data)
    producer.flush()
    print(f"Message sent to Kafka topic '{topic}': {data}")

# Send the emission data to Kafka every second
for data in emission_data:
    send_to_kafka(producer, topic_name, data)
    time.sleep(1)

Message sent to Kafka topic 'emissions': {'2000': 7.3015565276, '2001': 7.3445498509, '2002': 7.5910820191, '2003': 7.6661065819, '2004': 7.9264551349, '2005': 8.022971702, '2006': 8.247301388, '2007': 8.3558799857, '2008': 8.4741724795, '2009': 8.4774061324, '2010': 8.3759737748, '2011': 8.4235682823, '2012': 8.7662938718, '2013': 9.291035915, '2014': 9.8128903391, '2015': 10.1173860799, '2016': 10.4965465177, '2017': 10.8414736715, '2018': 10.6853845334, '2019': 10.6693128165, '2020': 10.9580914025, '2021': 11.1264878349, '2022': 11.0597619728}


In [38]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, DoubleType

# Initialize Spark session
spark = SparkSession.builder \
    .appName("BDENG8 Emission Project") \
    .getOrCreate()

# Define schema for the emission data
schema = StructType([
    StructField("2000", DoubleType(), True),
    StructField("2001", DoubleType(), True),
    StructField("2002", DoubleType(), True),
    StructField("2003", DoubleType(), True),
    StructField("2004", DoubleType(), True),
    StructField("2005", DoubleType(), True),
    StructField("2006", DoubleType(), True),
    StructField("2007", DoubleType(), True),
    StructField("2008", DoubleType(), True),
    StructField("2009", DoubleType(), True),
    StructField("2010", DoubleType(), True),
    StructField("2011", DoubleType(), True),
    StructField("2012", DoubleType(), True),
    StructField("2013", DoubleType(), True),
    StructField("2014", DoubleType(), True),
    StructField("2015", DoubleType(), True),
    StructField("2016", DoubleType(), True),
    StructField("2017", DoubleType(), True),
    StructField("2018", DoubleType(), True),
    StructField("2019", DoubleType(), True),
    StructField("2020", DoubleType(), True),
    StructField("2021", DoubleType(), True),
    StructField("2022", DoubleType(), True)
])

# Read data from Kafka
kafka_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "172.29.16.101:9092") \
    .option("subscribe", "emissions") \
    .load()

# Parse the value column in Kafka messages
emissions_df = kafka_df.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), schema).alias("data")) \
    .select("data.*")

# Process and display the data
query = emissions_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination()

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: org.apache.spark.SparkException: Only one SparkContext should be running in this JVM (see SPARK-2243).The currently running SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand

In [36]:
spark.stop()

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("SaveJSONToSparkSQLTable") \
    .getOrCreate()

# Given JSON data
emission_data = [{"2000": 7.3015565276, "2001": 7.3445498509, "2002": 7.5910820191, "2003": 7.6661065819,
                  "2004": 7.9264551349, "2005": 8.022971702, "2006": 8.247301388, "2007": 8.3558799857,
                  "2008": 8.4741724795, "2009": 8.4774061324, "2010": 8.3759737748, "2011": 8.4235682823,
                  "2012": 8.7662938718, "2013": 9.291035915, "2014": 9.8128903391, "2015": 10.1173860799,
                  "2016": 10.4965465177, "2017": 10.8414736715, "2018": 10.6853845334, "2019": 10.6693128165,
                  "2020": 10.9580914025, "2021": 11.1264878349, "2022": 11.0597619728}]

# Convert JSON data to DataFrame
rows = [Row(**data) for data in emission_data]
df = spark.createDataFrame(rows)

# Show the DataFrame schema and data (optional)
df.printSchema()
df.show()

# Save DataFrame as SparkSQL Table
table_name = "emission_data_table"
df.write.saveAsTable(table_name)

# Verify the table is created
spark.sql(f"SELECT * FROM {table_name}").show()




root
 |-- 2000: double (nullable = true)
 |-- 2001: double (nullable = true)
 |-- 2002: double (nullable = true)
 |-- 2003: double (nullable = true)
 |-- 2004: double (nullable = true)
 |-- 2005: double (nullable = true)
 |-- 2006: double (nullable = true)
 |-- 2007: double (nullable = true)
 |-- 2008: double (nullable = true)
 |-- 2009: double (nullable = true)
 |-- 2010: double (nullable = true)
 |-- 2011: double (nullable = true)
 |-- 2012: double (nullable = true)
 |-- 2013: double (nullable = true)
 |-- 2014: double (nullable = true)
 |-- 2015: double (nullable = true)
 |-- 2016: double (nullable = true)
 |-- 2017: double (nullable = true)
 |-- 2018: double (nullable = true)
 |-- 2019: double (nullable = true)
 |-- 2020: double (nullable = true)
 |-- 2021: double (nullable = true)
 |-- 2022: double (nullable = true)



Py4JJavaError: An error occurred while calling o42.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (172.31.181.17 executor driver): java.io.IOException: Cannot run program "python3": CreateProcess error=2, Das System kann die angegebene Datei nicht finden
	at java.base/java.lang.ProcessBuilder.start(ProcessBuilder.java:1128)
	at java.base/java.lang.ProcessBuilder.start(ProcessBuilder.java:1071)
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:181)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.

In [2]:
# Stop the Spark session
spark.stop()